In [ ]:
from DataIngestor import DataIngestor
from SharpeSurfaceGenerator import SharpeSurfaceGenerator

In [ ]:
import pandas as pd

In [ ]:
from datetime import datetime
start_year = 2018
start_date= datetime(start_year, 1, 1)
#end_date= datetime(2022, 12, 31)
end_date= datetime.now()
canada_etfs = ["FDN.TO",
"FBT.TO",
"ZEO.TO",
"SKYY.TO",
"XHC.TO",
"FHH.TO",
"ZGEN.TO",
"EARK.NE",
"CIF.TO",
"FHG.TO",
"TRVL.TO",
"FHQ.TO",
"NXTG.TO",
"ZBK.TO",
"RUBY.TO",
"FLI.TO",
"XST.TO",
"COMM.TO",
"CHPS.TO",
"TGRE.TO",
"XGB.TO"]



canada_bonds = [
    "VSC.TO",
    "VSB.TO",
    "ZIC.TO",
    "HBB.TO",
    "TDB.TO",
    "XBB.TO",
    "XGB.TO"]

stock_list = canada_bonds

In [ ]:
data_ingestor = DataIngestor(stock_list, start_date, end_date)

In [ ]:
close_df, adj_close_df = data_ingestor.close_df_adj_close_df()

In [ ]:

sharpe_gen = SharpeSurfaceGenerator(close_df, adj_close_df)

In [ ]:
close_df.head()

In [ ]:
sharpe_gen.plot_sharpe_surface()

In [ ]:
close_df.iloc[-1]

In [ ]:
close_df.tail()

In [ ]:
"""
Bond parameters
holding = 18
lookback = 8"""

In [ ]:
import pandas as pd
dfP = pd.read_csv(
    "VSC.TO.VSB.TO.ZIC.TO.HBB.TO.TDB.TO.XBB.TO.XGB.TO.csv", parse_dates=["Date"]
)

In [ ]:
for i in dfP.columns:
    print(i)

In [ ]:
dfP.columns[1]

In [ ]:
dfAll_ranks.to_csv(r'Results\dfAll_ranks.csv', header = True, index=True, encoding='utf-8')

In [ ]:
dfChoice.to_csv(r'Results\dfChoice.csv', header = True, index=True, encoding='utf-8')

In [ ]:
dfTJoin.to_csv(r'Results\dfTJoin.csv', header = True, index=True, encoding='utf-8')

In [ ]:
%run Hierarchal_Clustering.py

From the hierarchal clustering analysis, it is best to choose 5 clusters. 

Cluster 1: 15,2,1,3,18,17,28,11,0,10,16,19,14,13,12,29

Cluster 2: 21,27,7,25,20,22,23

Cluster 3: 24

Cluster 4: 26

Cluster 5: 6,4,5,8,9

In [ ]:
Z

In [ ]:
main_df.head

In [ ]:
%run Data_Ingestion_Rot_Momentum.py